In [7]:
import nibabel as nib
import numpy as np
from scipy.ndimage import label
import os
import csv
from pathlib import Path

In [8]:
def separate_clusters(tmap_path, output_dir, threshold, extent_threshold=10):
    """
    Separates non-contiguous clusters in a T-map into individual NIfTI masks,
    applying a voxel count threshold (extent threshold).

    Parameters:
        tmap_path (str): Path to the input T-map NIfTI file.
        output_dir (str): Directory to save the output NIfTI masks.
        threshold (float): Threshold value to apply to the T-map.
        extent_threshold (int): Minimum number of voxels for a cluster to be retained.

    Returns:
        tuple: (list of cluster paths, affine transformation matrix)
    """
    # Load the T-map
    tmap_img = nib.load(tmap_path)
    tmap_data = tmap_img.get_fdata()

    # Apply threshold
    thresholded_data = tmap_data > threshold

    # Label clusters
    labeled_data, num_clusters = label(thresholded_data)
    print(f"Found {num_clusters} clusters (before extent threshold).")

    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save each cluster that meets the extent threshold
    cluster_paths = []
    retained_clusters = 0
    for cluster_id in range(1, num_clusters + 1):
        cluster_mask = (labeled_data == cluster_id).astype(np.int16)
        # Count the number of voxels in the cluster
        voxel_count = np.sum(cluster_mask)

        # Skip clusters that do not meet the extent threshold
        if voxel_count < extent_threshold:
            continue

        # Save the cluster mask
        cluster_img = nib.Nifti1Image(cluster_mask, affine=tmap_img.affine, header=tmap_img.header)
        output_path = os.path.join(output_dir, f"cluster_{retained_clusters + 1}.nii")
        nib.save(cluster_img, output_path)
        cluster_paths.append(output_path)
        retained_clusters += 1
        print(f"Saved cluster {retained_clusters} with {voxel_count} voxels to {output_path}")

    print(f"Retained {retained_clusters} clusters (after extent threshold).")

    # Return cluster paths and the affine matrix
    return cluster_paths, tmap_img.affine



In [9]:
def calculate_centroids_and_peaks(cluster_paths, tmap_path):
    """
    Calculates the centroid and peak coordinates of each cluster.

    Parameters:
        cluster_paths (list): List of paths to cluster NIfTI masks.
        tmap_path (str): Path to the original T-map NIfTI file.

    Returns:
        list: A list of dictionaries for each cluster containing centroid and peak coordinates.
    """
    # Load the T-map
    tmap_img = nib.load(tmap_path)
    tmap_data = tmap_img.get_fdata()

    results = []
    for cluster_path in cluster_paths:
        # Load the cluster mask
        cluster_img = nib.load(cluster_path)
        cluster_data = cluster_img.get_fdata()

        # Mask the T-map with the cluster mask
        weighted_data = cluster_data * tmap_data

        # Find voxel coordinates
        coords = np.array(np.nonzero(cluster_data))
        weights = weighted_data[cluster_data > 0]

        # Calculate the weighted centroid
        if weights.sum() > 0:
            centroid = np.average(coords, axis=1, weights=weights)
            # Transform centroid to world coordinates
            centroid_world = nib.affines.apply_affine(tmap_img.affine, centroid)
        else:
            centroid_world = (np.nan, np.nan, np.nan)  # In case of no weights

        # Find the peak coordinate (voxel with the highest T-statistic)
        peak_voxel_index = np.unravel_index(np.argmax(weighted_data, axis=None), cluster_data.shape)
        peak_world = nib.affines.apply_affine(tmap_img.affine, peak_voxel_index)

        # Store results
        results.append({
            "centroid": tuple(centroid_world),
            "peak": tuple(peak_world)
        })
    
    return results

# Updated to include peak T value


In [10]:
import csv
import numpy as np
import nibabel as nib
from pathlib import Path


# Define second-level analysis directory
second_level = Path('/home/neel/Documents/SPM_results/second_level')

for spm_dir in second_level.iterdir():
    if spm_dir.is_dir():
        roi_output_dir = spm_dir / 'roi'
        
        # Ensure the ROI directory is clean
        if roi_output_dir.exists():
            for file in roi_output_dir.iterdir():
                file.unlink()
        else:
            roi_output_dir.mkdir(parents=True)
        
        # Read threshold value
        u_threshold_path = spm_dir / 'u_threshold.txt'
        with open(u_threshold_path, 'r') as f:
            u_threshold = float(f.read().strip())
        
        # Define T-map path and save path
        tmap = spm_dir / 'spmT_0001.nii'
        save_path = roi_output_dir / 'cluster_centroids.csv'
        
        print(f"Processing T-map: {tmap}")
        print(f"Using threshold: {u_threshold}")
        print(f"Saving results to: {save_path}")
        
        # Process clusters
        cluster_paths, affine = separate_clusters(str(tmap), str(roi_output_dir), threshold=u_threshold, extent_threshold=20)
        results = calculate_centroids_and_peaks(cluster_paths, str(tmap))
        
        # Compute peak T-values and cluster extents
        tmap_img = nib.load(tmap)
        tmap_data = tmap_img.get_fdata()
        
        for i, (cluster_path, result) in enumerate(zip(cluster_paths, results)):
            peak_voxel = np.round(nib.affines.apply_affine(np.linalg.inv(tmap_img.affine), result["peak"])).astype(int)
            if (0 <= peak_voxel[0] < tmap_data.shape[0] and
                0 <= peak_voxel[1] < tmap_data.shape[1] and
                0 <= peak_voxel[2] < tmap_data.shape[2]):
                result["peak_t_value"] = tmap_data[tuple(peak_voxel)]
            else:
                result["peak_t_value"] = float('nan')
            
            # Compute cluster extent
            cluster_img = nib.load(cluster_path)
            cluster_data = cluster_img.get_fdata()
            result["extent"] = int(np.sum(cluster_data > 0))
        
        # Sort results by peak T-value in descending order
        sorted_results = sorted(results, key=lambda x: x['peak_t_value'], reverse=True)
        
        # Reassign cluster numbers in order
        for i, result in enumerate(sorted_results, start=1):
            result['Cluster #'] = i
        
        # Print sorted results
        for result in sorted_results:
            print(f"Cluster {result['Cluster #']}:")
            print(f"  Centroid = {[round(coord) for coord in result['centroid']]}")
            print(f"  Peak = {[round(coord) for coord in result['peak']]}")
            print(f"  Peak T-value = {round(result['peak_t_value'], 2)}")
            print(f"  Extent = {result['extent']} voxels")
        
        # Save sorted results to CSV
        with open(save_path, 'w', newline='') as csvfile:
            fieldnames = ['Cluster #', 'Centroid', 'Peak', 'Peak T-value', 'Extent']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            for result in sorted_results:
                writer.writerow({
                    'Cluster #': result['Cluster #'],
                    'Centroid': [round(coord) for coord in result['centroid']],
                    'Peak': [round(coord) for coord in result['peak']],
                    'Peak T-value': round(result['peak_t_value'], 2),
                    'Extent': result['extent']
                })
        
        print(f"Processing completed for {spm_dir}\n")


Processing T-map: /home/neel/Documents/SPM_results/second_level/SPM_syllables_guslatho_syllableZipf_II/spmT_0001.nii
Using threshold: 7.89
Saving results to: /home/neel/Documents/SPM_results/second_level/SPM_syllables_guslatho_syllableZipf_II/roi/cluster_centroids.csv
Found 2 clusters (before extent threshold).
Saved cluster 1 with 902 voxels to /home/neel/Documents/SPM_results/second_level/SPM_syllables_guslatho_syllableZipf_II/roi/cluster_1.nii
Saved cluster 2 with 691 voxels to /home/neel/Documents/SPM_results/second_level/SPM_syllables_guslatho_syllableZipf_II/roi/cluster_2.nii
Retained 2 clusters (after extent threshold).
Cluster 1:
  Centroid = [-61, -17, 2]
  Peak = [-64, -12, -2]
  Peak T-value = 12.26
  Extent = 902 voxels
Cluster 2:
  Centroid = [62, -11, 1]
  Peak = [64, -4, -4]
  Peak T-value = 11.35
  Extent = 691 voxels
Processing completed for /home/neel/Documents/SPM_results/second_level/SPM_syllables_guslatho_syllableZipf_II

Processing T-map: /home/neel/Documents/SPM_

In [11]:


# spm_dir = '/home/neel/Documents/SPM_results/second_level/SPM-A_II_syllables_IPA_eSpeak_ijfix2'
# roi_output_dir = str(Path(spm_dir) / 'roi')




# if not os.path.exists(roi_output_dir):
#     os.makedirs(roi_output_dir)
# tmap = str(Path(spm_dir) / 'spmT_0001.nii')
# print(tmap)
# save_path = str(Path(roi_output_dir) / 'cluster_centroids.csv')
# print(save_path)

In [12]:
# tmap

In [13]:
# #delete all the contents of the roi_output_dir before running the code
# for file in os.listdir(roi_output_dir):
#     os.remove(os.path.join(roi_output_dir, file))
# cluster_paths, affine = separate_clusters(tmap, roi_output_dir, 5.409,extent_threshold=20)
# results = calculate_centroids_and_peaks(cluster_paths, tmap)


In [14]:

# # Calculate peak T-values and store results
# sorted_results = []
# for i, result in enumerate(results, start=1):
#     centroid = [round(coord) for coord in result["centroid"]]
#     peak = [round(coord) for coord in result["peak"]]
    
#     # Load the T-map to get the peak T-value
#     tmap_img = nib.load(tmap)
#     tmap_data = tmap_img.get_fdata()
    
#     # Convert the peak coordinate from world space to voxel space
#     peak_voxel = np.round(nib.affines.apply_affine(np.linalg.inv(tmap_img.affine), result["peak"])).astype(int)
    
#     # Ensure the voxel indices are within bounds
#     if (0 <= peak_voxel[0] < tmap_data.shape[0] and
#         0 <= peak_voxel[1] < tmap_data.shape[1] and
#         0 <= peak_voxel[2] < tmap_data.shape[2]):
#         peak_t_value = tmap_data[tuple(peak_voxel)]
#     else:
#         peak_t_value = float('nan')  # Out of bounds, assign NaN

#     # Calculate cluster size in voxels
#     cluster_path = cluster_paths[i - 1]
#     cluster_img = nib.load(cluster_path)
#     cluster_data = cluster_img.get_fdata()
#     cluster_size = int(np.sum(cluster_data > 0))  # Count non-zero voxels

#     sorted_results.append({
#         'Cluster #': i,
#         'Centroid': centroid,
#         'Peak': peak,
#         'Peak T-value': peak_t_value,
#         'Extent': cluster_size
#     })

# # Sort results by peak T-value in descending order
# sorted_results.sort(key=lambda x: x['Peak T-value'], reverse=True)

# # Reassign cluster numbers in order
# for i, result in enumerate(sorted_results, start=1):
#     result['Cluster #'] = i

# # Print sorted results
# for result in sorted_results:
#     print(f"Cluster {result['Cluster #']}:")
#     print(f"  Centroid = {result['Centroid']}")
#     print(f"  Peak = {result['Peak']}")
#     print(f"  Peak T-value = {round(result['Peak T-value'], 2)}")
#     print(f"  Extent = {result['Extent']} voxels")

# # Save sorted results to a CSV file
# with open(save_path, 'w', newline='') as csvfile:
#     fieldnames = ['Cluster #', 'Centroid', 'Peak', 'Peak T-value', 'Extent']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for result in sorted_results:
#         writer.writerow({
#             'Cluster #': result['Cluster #'],
#             'Centroid': result['Centroid'],
#             'Peak': result['Peak'],
#             'Peak T-value': round(result['Peak T-value'], 2),
#             'Extent': result['Extent']
#         })